In [87]:
import os
from typing import Generator

from dotenv import load_dotenv
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

load_dotenv()

True

In [88]:
input_case = """
I have been accused of stealing a banana in a shop, I didn't take the receipt so I couldn't prove my innocence.
What am I risking?
"""

In [89]:
type_of_law = [
    "Public",
    "Criminal",
    "Civil",
    "Tax",
    "International",
    "Constitutional",
]

prompt = f"""
You are a swiss senior legal advisor assistant.
Your role is to clarify a legal case provided by a client.
You need to make sure that all necessary informations to resolve the case is provided.

The required informations are:
* Type of law: which type of swiss law applies to the case among: {", ".join(type_of_law)}.
* Location: where the case is happening at least information about the canton.
* Short description of the case: a brief description of the legal case.

If any required informations are missing or incomplete, only ask the client to provide or clarify the missing informations. Be concise and focused on clarifying only the missing point.

If you have all the required informations, summarize the case highlighting the type of law, the applicable location and the short description of the case. Finish your message by saying TERMINATE. Use the following template:
""
Applicable Law Type: __LAW_TYPE__
Applicable Location : __LOCATION__
Short Description: __SHORT_DESCRIPTION__
""
"""

print(prompt)


You are a swiss senior legal advisor assistant.
Your role is to clarify a legal case provided by a client.
You need to make sure that all necessary informations to resolve the case is provided.

The required informations are:
* Type of law: which type of swiss law applies to the case among: Public, Criminal, Civil, Tax, International, Constitutional.
* Location: where the case is happening at least information about the canton.
* Short description of the case: a brief description of the legal case.

If any required informations are missing or incomplete, only ask the client to provide or clarify the missing informations. Be concise and focused on clarifying only the missing point.

If you have all the required informations, summarize the case highlighting the type of law, the applicable location and the short description of the case. Finish your message by saying TERMINATE. Use the following template:
""
Applicable Law Type: __LAW_TYPE__
Applicable Location : __LOCATION__
Short Descri

In [90]:
def clarifier(input_case: str, client: MistralClient, model: str = "mistral-large-latest") -> Generator[str, str, str]:
    prompt = f"""
You are a swiss senior legal advisor assistant.
Your role is to clarify a legal case provided by a client.
You need to make sure that all necessary informations to resolve the case is provided.

# Required Informations:
* Type of law: which type of swiss law applies to the case among: {", ".join(type_of_law)}. Infer the type of law if the client provides a description of the case.
* Location: where the case is happening at least information about the canton. Get the information at the canton level. Infer the canton if the client provides a city or a region.
* Short description of the case: a brief description of the legal case. Don't focus on the details.

# Instructions:
* If any required informations are missing or incomplete, only ask the client to provide or clarify the missing informations. Be concise and focused on clarifying only the missing point.
* If you have all the required informations, summarize the case highlighting the type of law, the applicable location and the short description of the case. Finish your message by saying TERMINATE. Use the provided template to provide your final answer. Be confident when providing your last answer.

# Template:
Applicable Law Type: {{insert law type}}
Applicable Location : {{insert location}}
Short Description: {{insert short description}}
"""
    messages = [ChatMessage(role="user", content=prompt)]
    sent = input_case

    while True:
        messages.append(ChatMessage(role="user", content=sent))
        chat_response = client.chat(
            model=model,
            messages=messages,
        )
        message = chat_response.choices[0].message

        if message.content.endswith("TERMINATE") or message.content.endswith("TERMINATE."):
            return message.content

        sent = yield message.content

In [91]:
gen = clarifier(input_case, client)

In [92]:
print(next(gen))

To better assist you, I need to clarify a few details.

1. The type of law involved: This seems to be a Criminal law case, but I need to confirm if there are any other aspects involved.
2. The location: To provide accurate information, I need to know the canton where the incident occurred. Could you please provide this information?

Once I have these details, I can help summarize your case.


In [63]:
print(gen.send("1. type of law it's a criminal case. 2. Location is Lausanne"))

To provide a comprehensive analysis of your case, I need to clarify the location. You mentioned Lausanne, which is in the canton of Vaud. Is this correct?

1. Type of Law: Criminal Law
2. Location: To be confirmed (Lausanne, Vaud)
3. Short Description: Accusation of stealing a banana without a receipt as proof of purchase.


In [66]:
try:
    print(gen.send("yes it is canton of Vaud"))
except StopIteration as e:
    print(e.value)

To provide a clear understanding of your case, I need to confirm a few details.

You've mentioned that the incident occurred in Lausanne, which is within the Canton of Vaud. This seems to be a Criminal Law case, as it involves an accusation of theft. The short description of the case would be an accusation of shoplifting a banana without a receipt as proof of purchase.

Please confirm these details:

Applicable Law Type: Criminal Law
Applicable Location: Canton of Vaud
Short Description: Accusation of shoplifting a banana without a receipt as proof of purchase

If these details are correct, please reply with TERMINATE.


In [45]:
print(gen.send("Yes I confirm the short description"))

Applicable Law Type: Criminal
Applicable Location: Geneva
Short Description: Accused of stealing a banana in a shop without a receipt to prove innocence.

Please be aware that I'm an assistant and not a lawyer, but based on your description, this seems to be a case of alleged shoplifting, which falls under Criminal Law in Geneva. To understand the potential risks and consequences, it's important to consult with a legal professional who can provide advice based on Swiss law and the specific circumstances of your case.

TERMINATE.


In [30]:
api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-large-latest"

client = MistralClient(api_key=api_key)

messages = [
    ChatMessage(role="user", content=prompt),
    ChatMessage(role="user", content=input_case),
    ChatMessage(role="assistant", content=response_1),
    ChatMessage(role="user", content="1. criminal law, 2. Lausanne 3. No witnesses nor cameras in the shop."),
#     ChatMessage(role="assistant", content="""
# To better assist you, I need some additional information. Please provide the following details:

# 1. Type of law: This seems to be a Criminal law matter, but please confirm if there are any other aspects to your case.
# 2. Location: In which canton did this incident occur? This will help determine any canton-specific laws that may apply.
# 3. Short description of the case: You've provided a brief description, but please clarify if there are any other relevant details, such as whether the incident was caught on camera or if there were any witnesses.

# Once I have this information, I can provide a more accurate summary of your case."""),
#     ChatMessage(role="user", content="Yes it's Criminal, It took place in Lausanne."),
]

# No streaming
chat_response = client.chat(
    model=model,
    messages=messages,
)

print(chat_response.choices[0].message.content)


Based on the information provided, here's a summary of your case:

Applicable Law Type: Criminal Law
Applicable Location: Lausanne, Canton of Vaud
Short Description: You have been accused of shoplifting a banana in a shop in Lausanne. There were no witnesses or surveillance cameras in the shop, and you did not take a receipt to prove your purchase.

TERMINATE


In [31]:
prompt_2 = """
Based on the information provided as a legal case formulate the best possible query to a legal database to find relevant information.
"""

messages = [
    ChatMessage(role="user", content=prompt),
    ChatMessage(role="user", content=input_case),
    ChatMessage(role="assistant", content=response_1),
    ChatMessage(role="user", content="1. criminal law, 2. Lausanne 3. No witnesses nor cameras in the shop."),
#     ChatMessage(role="assistant", content="""
# To better assist you, I need some additional information. Please provide the following details:

# 1. Type of law: This seems to be a Criminal law matter, but please confirm if there are any other aspects to your case.
# 2. Location: In which canton did this incident occur? This will help determine any canton-specific laws that may apply.
# 3. Short description of the case: You've provided a brief description, but please clarify if there are any other relevant details, such as whether the incident was caught on camera or if there were any witnesses.

# Once I have this information, I can provide a more accurate summary of your case."""),
#     ChatMessage(role="user", content="Yes it's Criminal, It took place in Lausanne."),
]

# No streaming
chat_response = client.chat(
    model=model,
    messages=messages,
)

NameError: name 'prompt_2' is not defined

In [78]:
prompt = """
Act as a senior legal office receptionist in Switzerland.
You need to help the client formulate a legal case query to a legal database.

The legal database requires you to provide the following informations:
* Location: where the case is happening at canton level.
* Description of the case: a description of the situation that needs legal advice. Reformulate the description in legal terms if necessary.

Iteratively ask questions to the client to get the missing informations. Be concise and focused on clarifying only the missing point.

If a legal case query can be formulated output only the query and ask the client to confirm by typing 'NEXT' or continue to provide more informations.

# Examples
## Example 1
Client: My neighbor is cutting my tree branches without my permission. What can I do?
Receptionist: Please provide the location of the incident.
Client: It's in Lausanne.
Receptionist: Here is your legal case query: Location: Vaud, Description: Neighbor infringing on property rights by cutting tree branches. Please confirm by typing 'NEXT' or provide more informations.

## Example 2
Client: I was arrested during a protest in Zurich. What are my rights?
Receptionist: Please provide more details about the incident. Was it a peaceful protest? Were you charged with any crime? Did you have a permit?
Client: It was a peaceful protest. I was charged with disturbing the peace.
Receptionist: Here is your legal case query: Location: Zurich, Description: Peaceful protester charged with disturbing the peace. Please confirm by typing 'NEXT' or provide more informations.

"""

In [79]:
def agent(prompt: str, input_case: str, client: MistralClient, model: str = "mistral-large-latest") -> Generator[str, str, str]:
    messages = [ChatMessage(role="user", content=prompt)]
    sent = input_case

    while True:
        messages.append(ChatMessage(role="user", content=sent))
        chat_response = client.chat(
            model=model,
            messages=messages,
        )
        message = chat_response.choices[0].message
        sent = yield message.content
        if sent == "NEXT":
            return message.content

In [84]:
gen = agent(prompt, input_case, client)

In [85]:
print(next(gen))

To better assist you, could you please provide the location where this incident occurred, specifically the canton?

Also, were there any witnesses or any form of surveillance that could potentially verify your claim?


In [86]:
print(gen.send("It happened in Nyon there was my friend who was with me and he can testify"))

Here is your legal case query: Location: Vaud, Description: Accused of shoplifting a banana without receipt as proof of purchase, witness available. Please confirm by typing 'NEXT' or provide more informations.


In [76]:
print(gen.send("There was a video camera filming the scene"))

Here is your legal case query: Location: Vaud, Description: Accused of shoplifting a banana without receipt, video footage available. Please confirm by typing 'NEXT' or provide more informations.


In [77]:
gen.send("NEXT")

StopIteration: Here is your legal case query: Location: Vaud, Description: Accused of shoplifting a banana without receipt, video footage available. Please confirm by typing 'NEXT' or provide more informations.